In [ ]:
import sys
sys.path.append('../python')
from get_vs import *
from misc import *
from models import *
from mcmc import *

import subprocess
import pickle
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
from tqdm import tqdm
import xarray as xr
import gcsfs
from dask_kubernetes import KubeCluster
from dask.distributed import Client

%matplotlib inline

In [ ]:
is_pangeo_data = True # True if in Pangeo binder, False if in laptop
if is_pangeo_data:
    cluster = KubeCluster(n_workers=10)
    client = Client(cluster)

The coordinates of the virtual stations in [Hydroweb](http://hydroweb.theia-land.fr) don't match with the rivers in [HydroSHEDS](http://www.hydrosheds.org). In order to find the corresponding coordinates in HydroSHEDS, we look around the original position for the pixel with the biggest accumulated flow which is bigger than a minimum flow. If no such flow is found, we look further around, until we find one (but not too far away, in which case we just drop the virtual station). The new_lat/new_lon are the coordinates of this pixel, if found.

In [ ]:
if not os.path.exists('../data/amazonas/amazonas.pkl'):
    df = locate_vs('../data/amazonas/amazonas.txt', pix_nb=20, acc_min=1_000_000)
    df.to_pickle('../data/amazonas/amazonas.pkl')
else:
    df = pd.read_pickle('../data/amazonas/amazonas.pkl')

In [ ]:
sub_latlon = df[['new_lat', 'new_lon']].dropna().values
print(f'Out of {len(df)} virtual stations in Hydroweb, {len(sub_latlon)} could be found in HydroSHEDS.')

The following coordinates are duplicated because some virtual stations fall inside the same pixels.

In [ ]:
rm_latlon = [(-4.928333333333334, -62.733333333333334), (-3.8666666666666667, -61.6775)]

In [ ]:
df_ll = df[['new_lat', 'new_lon']].dropna()
duplicated = df_ll[df_ll.duplicated(keep=False)]
duplicated

In [ ]:
# all the subbasins in the hydrologic partition (including virtual stations)
#gcs_get_dir('pangeo-data/gross/ws_mask/amazonas', 'ws_mask/amazonas', fs)
#gcs_w_token = gcsfs.GCSFileSystem(project='pangeo-data', token='browser')
if is_pangeo_data:
    fs = gcsfs.GCSFileSystem(project='pangeo-data')
    labels = [os.path.basename(path[:-1]) for path in fs.ls('pangeo-data/gross/ws_mask/amazonas')]
else:
    labels = os.listdir('ws_mask/amazonas')
print('Total number of subbasins:', len(labels))

In [ ]:
label_pickle_path = '../data/amazonas/labels.pkl'
if not os.path.exists(label_pickle_path):
    labels_without_vs = list(labels)
    labels_with_vs = {}
    for label in tqdm(labels):
        ds = xr.open_zarr(f'ws_mask/amazonas/{label}')
        da = ds['mask']
        olat, olon = da.attrs['outlet']
        idx = df_ll[(olat-0.25/1200<df_ll.new_lat.values) & (df_ll.new_lat.values<olat+0.25/1200) & (olon-0.25/1200<df_ll.new_lon.values) & (df_ll.new_lon.values<olon+0.25/1200)].index.values
        if len(idx) > 0:
            labels_without_vs.remove(label)
            labels_with_vs[label] = list(df.iloc[idx].station.values)
    with open(label_pickle_path, 'wb') as f:
        pickle.dump((labels_with_vs, labels_without_vs), f)
else:
    with open(label_pickle_path, 'rb') as f:
        labels_with_vs, labels_without_vs = pickle.load(f)

In [ ]:
labels_with_vs_tree = get_label_tree(list(labels_with_vs))

In [ ]:
da_mask = get_mask('ws_mask/amazonas', labels)
da_mask.to_dataset(name='mask').to_zarr('mask')

In [ ]:
d0, d1 = '2000-03-01 12:00:00', '2018-12-31'
precipitation = get_precipitation(d0, d1, 'mask', labels)

In [ ]:
pet = get_pet(d0, d1, 'mask')

In [ ]:
df = DataFrame()
df['p'] = precipitation
df['e'] = pet
df.plot(figsize=(15, 5))

In [ ]:
d0, d1 = '2000-03-01 12:00:00', '2018-12-31'
x_start = [100, 0, 10, 1]
x_range = ((0.1, 1e4), (-1, 1), (0.1, 1e3), (0.1, 1e2))
sample_nb = 2_000 # number of samples generated by MCMC
burnin = sample_nb // 10 # number of burnin samples
warmup = 30 * 24 * 2

q_ensemble = {}
#for ws_i in range(ws_nb):
for label in labels_with_vs_tree:
    # get basin's labels
    ws_labels = startswith_label(label, labels)
    # get basin's mask
    os.makedirs('ws_mask/amazonas', exist_ok=True)
    for l in ws_labels:
        if not os.path.exists(f'ws_mask/amazonas/{l}'):
            fs = gcsfs.GCSFileSystem(project='pangeo-data')
            gcs_get_dir(f'pangeo-data/gross/ws_mask/amazonas/{l}', f'ws_mask/amazonas/{l}', fs)
    da_mask = get_mask('ws_mask/amazonas', ws_labels)
    subprocess.check_call('rm -rf mask'.split())
    da_mask.to_dataset(name='mask').to_zarr('mask')
    # get basin's precipitation and PET, and water level at virtual station
    if is_pangeo_data:
        p = get_precipitation(d0, d1, 'mask')
        e = get_pet(d0, d1, 'mask')
    else:
        peq = pd.read_pickle('peq.pkl')
        p = peq.p
        e = peq.e
    he = get_waterlevel(d0, d1, labels_with_vs[label][0]) # there might be several stations
    peq = DataFrame()
    peq['p'] = p
    peq['e'] = e
    peq['h_obs'] = he.h
    peq['h_err'] = he.e
    is_source_basin = True
    if is_source_basin:
        area_head = 1
        area_tail = 0
        # prior probability distribution is uniform for head basin
        x0 = x_start
        x_prior = [uniform_density(*r) for r in x_range]
        lnprob_prior = [lnprob_from_density(p, *r) for p, r in zip(x_prior, x_range)]
    else:
        area_head = ws_i
        area_tail = 1
        x0 = [xy[0][np.argmax(xy[1])] for xy in x_head]
        x0 += [d_start] + x_start
        # prior probability distribution is uniform for tail basin
        lnprob_prior = [lnprob_from_density(p, *r) for p, r in zip(x_head, x_range)]
        lnprob_prior += [lnprob_from_density(uniform_density(*d_range), *d_range)]
        x_tail = [uniform_density(*r) for r in x_range]
        lnprob_prior += [lnprob_from_density(p, *r) for p, r in zip(x_tail, x_range)]
    lnprob = get_lnprob(gr4hh, warmup, peq, lnprob_prior, area_head, area_tail)
    # run MCMC
    futures = [client.submit(mcmc.Sampler, x0, lnprob, actor=True) for _ in range(10)]
    samplers = [future.result() for future in futures]
    futures = [sampler.run(sample_nb, burnin) for sampler in samplers]
    samples2, q_sim2 = [future.result() for future in futures]
    sys.exit()
    #sampler = Sampler(x0, lnprob)
    #samples, q_sim = sampler.run(sample_nb, burnin)
    # get simulated streamflow and uncertainty
    q_sim = np.array(q_sim)
    q_ensemble[f'f{ws_i}'] = q_sim
    # plot updated streamflow
    #plot_series(ensemble=q_sim[:, -days:], true=df[f'q_true_{ws_i}'].values[-days:], title=f'Streamflow at the outlet of $B_{ws_i}$')
    if False:#(ws_i > 0) and (ws_i < ws_nb - 1):
        # reduce dual model to single model
        peq = df[['p', 'e']]
        x_prior = [uniform_density(*r) for r in x_range]
        lnprob_prior = [lnprob_from_density(p, *r) for p, r in zip(x_prior, x_range)]
        q_kde = np.empty((2, n_kde, q_sim.shape[1]))
        for i in range(q_kde.shape[2]):
            q_kde[:, :, i] = get_kde(q_sim[:, i], nb=n_kde)
        lnprob = get_lnprob(peq, lnprob_prior, 1, 0, q_kde)
        x0 = x_start
        sampler = mcmc.Sampler(x0, lnprob)
        samples, q_sim = sampler.run(sample_nb, burnin)
    x_head = [get_kde(samples[:, i]) for i in range(4)]

In [ ]:
def get_waterlevel(from_time, to_time, vs):
    url = f'http://hydroweb.theia-land.fr/hydroweb/view/{vs}?lang=en&basin=AMAZONAS'
    df, _, _ = get_vs(url)
    # don't bother about 15min offset, nor about multiple values for a date
    he = df.loc[from_time:to_time]
    he.index = he.index.round('30min')
    he = he.resample('30min').asfreq()
    he.index = he.index + timedelta(minutes=15)
    return he

In [ ]:
def get_lnprob(gr4, warmup, peq, lnprob_prior, area_head, area_tail, q_kde=None):
    def lnprob(x):
        lnp = 0
        for i, v in enumerate(x):
            lnp += lnprob_prior[i](v)
        if not np.isfinite(lnp):
            return -np.inf, np.ones_like(peq.p.values) * np.inf

        x_head = x[:5] # this includes the delay in the gr4 model
        g_head = gr4(x_head)
        if area_tail > 0:
            x_tail = x[5:]
            g_tail = gr4(x_tail)
            q_tail = g_tail.run([peq.p.values, peq.e.values])
        else:
            q_tail = 0
        q_sim = (g_head.run([peq.p.values, peq.e.values]) * area_head + q_tail * area_tail) / (area_head + area_tail)
        if q_kde is None:
            # observation is measured water level
            h_obs = peq.h_obs.values
            h_err = peq.h_err.values
            h_sim = np.hstack((np.full(warmup, np.nan), dist_map(q_sim[warmup:], h_obs[warmup:])))
            df = DataFrame({'h_sim': h_sim, 'h_obs': h_obs, 'h_err': h_err})[warmup:].dropna()
            std2 = df.h_err * df.h_err
            return lnp + np.sum(-np.square(df.h_sim.values - df.h_obs.values) / (2 * std2) - np.log(np.sqrt(2 * np.pi * std2))), q_sim
        else:
            # observation is simulated streamflow
            lnp_q = 0
            for i in range(warmup, q_sim.size, sim_step):
                lnp_q += lnprob_from_density(q_kde[:, :, i])(q_sim[i])
            return lnp + lnp_q, q_sim
    return lnprob